# [Issue #1384](https://github.com/JetBrains/lets-plot/issues/1384): geomRibbon with multiple colors doesn't look as expected

In [1]:
from math import sin

import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
def get_data(n):
    x = list(range(n))
    a = [sin(v / 10.0) * 20.0 for v in x]
    b = [sin(v / 6.0 + 1.0) * 20.0 - 2.0 for v in x]
    return {
        "x": x,
        "min": [min(first, second) for (first, second) in zip(a, b)],
        "max": [max(first, second) for (first, second) in zip(a, b)],
        "label": [("a" if first > second else "b") for (first, second) in zip(a, b)],
    }

data = get_data(60)

## Plot from the issue

In [4]:
ggplot(data) + \
    geom_ribbon(aes(x="x", ymin="min", ymax="max", fill="label", color="label"), alpha=.2)

## Explore the data

In [5]:
ggplot(data) + \
    geom_text(aes("x", "min", label="label"), color="blue", manual_key="min") + \
    geom_text(aes("x", "max", label="label"), color="red", manual_key="max")

## Expected?

In [6]:
ggplot(data) + \
    geom_vline(aes(xintercept="x", color="label"), alpha=.2, size=3.5) + \
    geom_ribbon(aes(x="x", ymin="min", ymax="max"))

In [7]:
def get_raw_df(n):
    x = list(range(n))
    a = [sin(v / 10.0) * 20.0 for v in x]
    b = [sin(v / 6.0 + 1.0) * 20.0 - 2.0 for v in x]
    df = pd.DataFrame({
        "x": x,
        "a": a,
        "b": b,
        "min": [min(first, second) for (first, second) in zip(a, b)],
        "max": [max(first, second) for (first, second) in zip(a, b)],
    })
    gt_s = df["a"].gt(df["b"])
    return df.assign(
        label=gt_s.ne(gt_s.shift()).cumsum().astype(str)
    )

raw_df = get_raw_df(60)
raw_df.head(10)

,x,a,b,min,max,label
0,0,0.000000,14.829420,0.000000,14.829420,1
1,1,1.996668,16.388900,1.996668,16.388900,1
2,2,3.973387,17.438758,3.973387,17.438758,1
3,3,5.910404,17.949900,5.910404,17.949900,1
4,4,7.788367,17.908159,7.788367,17.908159,1
5,5,9.588511,17.314693,9.588511,17.314693,1
6,6,11.292849,16.185949,11.292849,16.185949,1
7,7,12.884354,14.553207,12.884354,14.553207,1
8,8,14.347122,12.461718,12.461718,14.347122,2
9,9,15.666538,9.969443,9.969443,15.666538,2


## First try: correct the `"label"` column in the data and use `scale_manual()`

In [8]:
ggplot(raw_df) + \
    geom_ribbon(aes(x="x", ymin="min", ymax="max", color="label", fill="label"), alpha=.2) + \
    scale_manual(["color", "fill"], values=["red", "blue"])

In [9]:
def get_continuous_df(df, target_col):
    cols = df.columns
    out_rows = []
    for i in range(len(df)):
        if i > 0 and df[target_col].iloc[i] != df[target_col].iloc[i-1]:
            boundary = df.iloc[i].copy()
            boundary[target_col] = df[target_col].iloc[i-1]
            out_rows.append(boundary)
        out_rows.append(df.iloc[i])
    return pd.DataFrame(out_rows, columns=cols).reset_index(drop=True)

df = get_continuous_df(raw_df, "label")
df.head(10)

,x,a,b,min,max,label
0,0,0.000000,14.829420,0.000000,14.829420,1
1,1,1.996668,16.388900,1.996668,16.388900,1
2,2,3.973387,17.438758,3.973387,17.438758,1
3,3,5.910404,17.949900,5.910404,17.949900,1
4,4,7.788367,17.908159,7.788367,17.908159,1
5,5,9.588511,17.314693,9.588511,17.314693,1
6,6,11.292849,16.185949,11.292849,16.185949,1
7,7,12.884354,14.553207,12.884354,14.553207,1
8,8,14.347122,12.461718,12.461718,14.347122,1
9,8,14.347122,12.461718,12.461718,14.347122,2


## Second try: add continuity

In [10]:
ggplot(df) + \
    geom_ribbon(aes(x="x", ymin="min", ymax="max", color="label", fill="label"), alpha=.2) + \
    scale_manual(["color", "fill"], values=["red", "blue"])